In [ ]:
import optuna
import numpy as np
import matplotlib.pyplot as plt
import os
work_dir = "H:\workspace\ptyrad"
os.chdir(work_dir)
print("Current working dir: ", os.getcwd())

In [ ]:
import torch

In [ ]:
main_folder = 'H:/workspace\ptyrad\output\paper/tBL_WSe2/20241201_hypertune_GridSampler_benchmark_1slice/'
subdir = 't0021_full_N16384_dp128_flipT100_random128_p1_1obj_1slice_Adam_plr1e-4_oalr5e-4_oplr5e-4_slr5e-4_orblur0.2_ozblur1_oathr0.98_opos_sng1.0_spr0.1/'
model_path = main_folder + subdir + 'model_iter0200.pt'
model = torch.load(model_path)

In [ ]:
np.mean(model['iter_times'])

In [ ]:
np.std(model['iter_times'])

In [ ]:
# You can use Matplotlib instead of Plotly for visualization by simply replacing `optuna.visualization` with
# `optuna.visualization.matplotlib` in the following examples.
# from optuna.visualization.matplotlib import plot_contour
# from optuna.visualization.matplotlib import plot_edf
# from optuna.visualization.matplotlib import plot_intermediate_values
# from optuna.visualization.matplotlib import plot_optimization_history
# from optuna.visualization.matplotlib import plot_parallel_coordinate
# from optuna.visualization.matplotlib import plot_param_importances
# from optuna.visualization.matplotlib import plot_rank
# from optuna.visualization.matplotlib import plot_slice
# from optuna.visualization.matplotlib import plot_timeline

from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline

In [ ]:
storage_path = "sqlite:///hypertune.sqlite3"
study_names = optuna.study.get_all_study_names(storage=storage_path)
print(f"study_names = {study_names}")
studies = {name: optuna.load_study(study_name=name, storage=storage_path) for name in study_names}

In [ ]:
# https://optuna.readthedocs.io/en/stable/reference/generated/optuna.importance.PedAnovaImportanceEvaluator.html
# The importance can be interpreted as how important each hyperparameter is to get the performance better than baseline.
# If evaluate on local, the importances imply how importance each parameter is during optimization. Meanwhile, evaluate_on_local=False gives the importances in the specified search_space.
study = studies['optimization_tBL_WSe2_random']
evaluator = optuna.importance.PedAnovaImportanceEvaluator(baseline_quantile=0.05, evaluate_on_local=True)
plot_param_importances(study, evaluator=evaluator)
plot_contour(study)
plot_intermediate_values(study)
plot_optimization_history(study)
plot_parallel_coordinate(study)